In [2]:
############################           PUBS AND TUBES          ############################


#this is version 1 of the pubs and tubes script


import pandas as pd
import numpy as np
import math
import missingno as mn
import matplotlib.pyplot as plt
import seaborn as sn
import pickle


#read in the pubs dataset already processed with distance to their nearest tube line attached
#easiest thing is probably to start adding total travel time to nearest node point of a different line
#again can cover that later, for now just take the closest one
#orrrr just treat them as independant pubs and double the length of your data, it wouldn't add much processing time

#jupyter commands dont work in actual scripts

pubswithdist = pd.read_csv('C:\\Users\\George\\Downloads\\MITM\\pubswithdist.csv',index_col=0)
#read in the already calculated tube network travel time data set

#data_tubetravel = pd.read_csv('C:\\Users\\George\\Downloads\\MITM\\data_tubetravel.csv',index_col=0)

with open('C:\\Users\\George\\Downloads\\MITM\\.pickle','rb') as f:
    data_tubetravel = pickle.load(f,encoding="utf8")


data_stations = pd.read_csv("C:\\Users\\George\\Downloads\\MITM\\stations_csv.sv.csv")
#now grab the all important travel times
data_travel = pd.read_csv("C:\\Users\\George\\Downloads\\MITM\\travel_times.csv")


#need to assign every pub a nearest tube
#need to define a function for distance between coordinates
#use the haversine formula
#need the angle in radians

def coords_to_distance(x1,x2,y1,y2):
    x1 = x1 * math.pi / 180
    x2 = x2 * math.pi / 180
    y1 = y1 * math.pi / 180
    y2 = y2 * math.pi / 180
    dlat = x2 - x1
    dlon = y2 - y1
    a = (math.pow(math.sin(dlat/2),2)) + math.cos(x1) * math.cos(x2) * (math.pow(math.sin(dlon/2),2))
    c = 2 * math.atan2(math.sqrt(a),math.sqrt(1-a))
    d = c * 6373
    #this is in km
    return d

def find_second_station(previous_station):
    alreadycheckedlines= []
    for line in data_travel[data_travel['station1'] == previous_station].iterrows():
        #print(line)
        alreadycheckedlines.append(line[1]['line'])
        datatravel_temp = data_travel[~data_travel['line'].isin(alreadycheckedlines)]
        data_filtered = data_stations[data_stations['id'].isin(datatravel_temp['station1'])]
    return data_filtered


#need this to be self contained, and output an identifiable dataset ready for further processing
def distance_to_pubs(input_lat,input_long,user_num,data_stations):
    min_distance = 1000
    closest_tube_id = []
    closest_tube = []
    for tube in data_stations.itertuples():
        #get distance between input coords and all tubes
        current_dist = coords_to_distance(getattr(tube,'latitude'),input_lat,getattr(tube,'longitude'),input_long)
        if current_dist < min_distance:
            min_distance = current_dist
            closest_tube_id = getattr(tube,'id')
            #closest_line = getattr(line, '')
            #this will return the closest tube station id to the input coords
            #could potentially be optimised further but not worth it because of how quick it currently is
            
    #this loop returns the closest tube station to the input coords
    #then the below works out the travel time to all the things from that
    #data tubetravel contains travel time from each tube station to every other tube station
    #so just filter to the nearest tube station and check time to the rest of the network
    #need to check each line as well
    
    x= 1
    person_travel = []
    tube_id = []
    tube_results = []
    for line in data_tubetravel[data_tubetravel['id'] == closest_tube_id]['travel_times']:
        while x < len(data_stations):
            person_travel.append(line[x]['time']) 
            tube_id.append(x)
            x += 1

    #the above returns the travel time to all stations from the nearest station
    user_num = str(user_num)    
    person_travel = pd.DataFrame(person_travel)
    tube_id = pd.DataFrame(tube_id)
    individ_tube = pd.merge(tube_id,person_travel,left_index = True, right_index = True)
    individ_tube.columns = ['tube_id','travel_time_to_tube'+user_num]
    
    
    full_travel = pd.merge(pubs_with_dist_filt,individ_tube, how = 'left',left_on = '0_y',right_on = 'tube_id')
    min_distance_time = min_distance / 0.084
    full_travel['total time' + user_num] = full_travel['traveltime_totube_pub'] + full_travel['travel_time_to_tube'+str(user_num)] + min_distance_time
    return full_travel,closest_tube_id




#could be worth filtering pubs with dist to only a couple of columns so that it's neater?
pubs_with_dist_filt = pubswithdist[['0_y','index','traveltime_totube_pub']]
numpeeps = 2
i = 1
while i <= numpeeps:
    full_travel_i_1, i_closest_tube_id = distance_to_pubs(51.5020275,-0.0267862,i,data_stations)
    full_travel_i_2, i_second_closest_tube = distance_to_pubs(51.5020275,-0.0267862,i,find_second_station(i_closest_tube_id))
    full_travel_i = pd.merge(full_travel_i_1,full_travel_i_2, left_index=True,right_index=True)
    #full_travel_i['True_shortest' + str(i)] = min(full_travel_i['total time' + str(i) + '_x'], full_travel_i['total time' + str(i) + '_y'])
    full_travel_i['true_shortest' + str(i)] = full_travel_i[['total time' + str(i) + '_x', 'total time' + str(i) + '_y']].min(axis=1)
    name = str(i)
    full_travel_i_final = full_travel_i[['true_shortest' + name]]
    if i > 1:
        full_current = pd.merge(full_travel_i_final,full_current,left_index=True,right_index=True)
    else:
        full_current = full_travel_i_final
    i += 1
#test_fulltravel,test_closest_tube_id = distance_to_pubs(51.5020275,-0.0267862,2,data_stations)
#this works fine, worth in your loop goingn twice for each person and checking the other nearest tube
#then combine those datasets, take only the shortest travel time to each pub and move on
#ok so once that works what needs to happen?

cols_to_use = ['true_shortest' + str(p+1) for p in range(numpeeps)]
#full_current['Total'] = full_current.loc[:, cols_to_use].sum(axis=1)
full_current['Total'] = full_current[cols_to_use].sum(axis =1 )
full_current = pd.merge(full_current,pubswithdist,left_index= True,right_index=True)
full_current = full_current[full_current['Total'] != 0]




In [11]:
finalfinal = full_current.sort_values('Total')
#finalfinal2 = pd.DataFrame(finalfinal)
#finalfinal_top = finalfinal.head(10)
#finalfinal_top = pd.DataFrame(finalfinal_top)
#finalfinal_top = finalfinal_top[['Total',2,8]]
print(finalfinal)

      true_shortest2  true_shortest1       Total  index     0_x  \
3314        7.386873        7.386873   14.773745  12600  140031   
3393        7.386873        7.386873   14.773745  12679  141942   
3376        7.558988        7.558988   15.117976  12662  141471   
3426        7.575081        7.575081   15.150163  12712  142252   
3455        7.612257        7.612257   15.224515  12741  142340   
3415        7.613817        7.613817   15.227634  12701  142226   
3339        7.613817        7.613817   15.227634  12625  140591   
3444        7.613817        7.613817   15.227634  12730  142300   
3452        9.534042        9.534042   19.068083  12738  142332   
3417       10.664390       10.664390   21.328781  12703  142231   
3378       11.592368       11.592368   23.184737  12664  141583   
3428       11.728108       11.728108   23.456216  12714  142271   
3461       12.999069       12.999069   25.998139  12747  142376   
3470       13.002284       13.002284   26.004569  12756  14251

In [20]:
finalfinal[['1']]

,1
3314,All Bar One
3393,Slug and Lettuce
3376,Nicolas
3426,The Henry Addington
3455,The Tea Merchant
3415,The Fine Line
3339,Davys At Canary Wharf
3444,The Pagination
3452,The Sipping Room
3417,The George
